##### Copyright 2022 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Parameter Sweeps

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/params"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/params.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/params.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/params.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

## Concept of Circuit Parameterization and Sweeps

Suppose one has a quantum circuit and in this circuit there is a gate with some parameter. One might wish to run this circuit for different values of this parameter.  An example of this type of setup is a Rabi flop experiment. In this experiment, one runs a set of quantum computations where one 1) starts in  $|0\rangle$ state, 2) rotates the state by $\theta$ about the $x$ axis, i.e. applies the gate $\exp(i \theta X)$, and 3) measures the state in the computational basis.  Running this experiment for multiple values of $\theta$, and plotting the probability of observing a $|1\rangle$ outcome yields the quintessential $\cos^2$ probability distribution as a function of the different parameters $\theta$.  To support this type of experiment, Cirq provides the concept of parameterized circuits and parameter sweeps.  

Let's illustrate parameter sweeps by a simple example.  Suppose that we want to compare two quantum circuits that are very similar except for a single gate.

In [ ]:
q0 = cirq.LineQubit(0)

circuit1 = cirq.Circuit([cirq.H(q0), cirq.Z(q0)**0.5, cirq.H(q0), cirq.measure(q0)])
print(f"circuit1:\n{circuit1}")

circuit2 = cirq.Circuit([cirq.H(q0), cirq.Z(q0)**0.25, cirq.H(q0), cirq.measure(q0)])
print(f"circuit2:\n{circuit2}")

One could run (either on hardware or in simulation) these circuits separately, for example, and collect statistics on the results of these circuits. However we can use parameter sweeps to do this in a cleaner and more perfomant manner.  

First one defines a parameter, and constructs a circuit that depends on this parameter. We use [SymPy](https://www.sympy.org/en/index.html){:external}, a symbolic mathematics package, to define our parameters.  For example, here we define a symbol, theta, and use it to construct a paremeterized circuit.

In [ ]:
import sympy

theta = sympy.Symbol("theta")

circuit = cirq.Circuit([cirq.H(q0), cirq.Z(q0)**theta, cirq.H(q0), cirq.measure(q0)])
print(f"circuit:\n{circuit}")

Notice now that our circuit contains a `cirq.Z` gate that is raised to a power, but this power is our parameter `theta`.  This is a "parameterized circuit".  An alternative way to construct this, where we see that the parameter is actually a parameter on the gate's constructor arguments, is

In [ ]:
circuit = cirq.Circuit([cirq.H(q0), cirq.ZPowGate(exponent=theta)(q0), cirq.H(q0), cirq.measure(q0)])
print(f"circuit:\n{circuit}")

We can check whether an object in Cirq is parameterized using `cirq.is_parameterized`:

In [ ]:
cirq.is_parameterized(circuit)

Parameterized circuits are just like normal circuits, they just aren't defined in terms of actually gates you can run on a quantum computer without the additional information about the values of the parameters.  Following along with our example above, we can generate the two circuits (`circuit1` and `circuit2`) by using `cirq.resolve_parameter` and supplying the parameters:

In [ ]:
# circuit1 has theta = 0.5
cirq.resolve_parameters(circuit, {"theta": 0.5})

More interestingly, can combine parameterized circuits with a list of parameter assignments when doing things like running circuits or simulating them.  These lists of parameter assignements are called "sweeps".  For example we can use a simulators `run_sweep` method to run simulations for the parameters corresonding to the two circuits defined above.

In [ ]:
sim = cirq.Simulator()
results = sim.run_sweep(circuit, repetitions=25, params=[{"theta": 0.5}, {"theta": 0.25}])
for result in results:
    print(f"param: {result.params}, result: {result}")

To recap, we can construct parameterized circuits which depend on parameters that have not yet been assigned a value.  These parameterized circuits can then be resolved to circuits with actual values via a dictionary that maps the parameter name to the value. We can also construct lists of dictionaries of parameter assignments, called sweeps, and pass this to many objects in Cirq that use circuits to do an action (such as simulate or run on hardware).  For each of the elements in the sweep, the object will do the action using the parameters as described by the element.

## Constructing Sweeps

Above we constructed a sweep by simply constructing a list of parameter assignments, `[{"theta": 0.5}, {"theta": 0.25}]`.  Cirq also provides other ways to construct sweeps.  

One useful method for constructing parameter sweeps is `cirq.Linspace` which creates a sweep over a list of equally spaced elements.  

In [ ]:
# Create a sweep over 5 equally spaced values from 0 to 2.5.
params = cirq.Linspace(key="theta", start=0, stop=2.5, length=5)
for param in params:
    print(param)

Many methods that take a sweepable will take a list, but if one want to construct an explicit sweepable from a list, `cirq.Points` does this.

In [ ]:
params = cirq.Points(key="theta", points=[0, 1, 3])
for param in params:
    print(param)

Often one wants to sweep over multiple parameters. Two common cases are that one wants to combined two sweeps over parameters to take all combinations of these parameters (the cartesian product), or taking combinations that match up elementwise (zipping). Here are two examples that show how to do this

In [ ]:
sweep1 = cirq.Linspace("theta", 0, 1, 5)
sweep2 = cirq.Points("gamma", [0, 3])
# By taking the product of these two sweeps, we can sweep over all possible
# combinations of the parameters.
for param in sweep1 * sweep2:
    print(param)

In [ ]:
sweep1 = cirq.Points("theta", [1, 2, 3])
sweep2 = cirq.Points("gamma", [0, 3, 4])
# By taking the sum of these two sweeps, we can combine the sweeps
# elementwise (similar to python's zip function):
for param in sweep1 + sweep2:
    print(param)

`cirq.Linspace` and `cirq.Points` are instances of the `cirq.Sweep` class, which explicitly supports cartesian product with the `*` operation, and zipping with the `+` operation. Other mathematical operations will not work in general *between sweeps*.

## Symbols and Expressions

Cirq uses Sympy to define its parameters. Sympy is a general symbolic mathematics toolset, and we can leverage this in Cirq.  For example, in Sympy, we can define an expression and use it to construct circuits that depend on this expression:

In [ ]:
# We construct an expression for 0.5 * a + 0.25:
expr = 0.5 * sympy.Symbol("a") + 0.25
print(expr)

In [ ]:
# We can use this in the circuit:
circuit = cirq.Circuit(cirq.X(q0)**expr, cirq.measure(q0))
print(f"circuit:\n{circuit}")

When we resolve parameters for this circuit, the expression will be evaluated

In [ ]:
print(cirq.resolve_parameters(circuit, {"a": 0}))

Similarly when we run a simulation, we can pass in a sweep, and Cirq will evaluate this expression for each of the possible values in the sweep:

In [ ]:
sim.run_sweep(circuit, repetitions=20, params=[{"a": 0}, {"a": 1}])

Sympy supports a large number of numeric functions and methods, and we can create fairly sophisticated expressions:

In [ ]:
print(sympy.cos(sympy.Symbol("a"))**sympy.Symbol("b"))

Cirq can evaluate numerically all of the expressions Sympy can evalute. One should note however, that if one is running a parameterized circuit on a service (such as on a hardware backed quantum computing service) these services may not suport evaluating all expressions. See documentation for the particular service for details. However, as a general workaround, one can instead use Cirq's flattening ability to evaluate the parameters server side.

### Flattening Expressions

Suppose we build a circuit with multiple expressions in the circuit:

In [ ]:
a = sympy.Symbol('a')
circuit = cirq.Circuit(
    cirq.X(q0) ** (a/4),
    cirq.Y(q0) ** (1-a/2),
    cirq.measure(q0)
)
print(circuit)

The idea behind flattening is that for each of the expressions used in the circuit, we create a new symbol for this expression, and then construct and object, a `cirq.ExpressionMap`, that has knowledge about how to map from the bare symbols to the value of the expression. 

In [ ]:
# Flatten returns two objects, the circuit with new symbols, and the mapping from old to new values.
c_flat, expr_map = cirq.flatten(circuit)
print(c_flat)
print(expr_map)

Notice that the new circuit has new symbols, `<a/2>` and `<1-a/2`. These are not expressions.  We can see this by looking at the value of the exponent in the first gate:

In [ ]:
first_gate = c_flat[0][q0].gate
print(first_gate.exponent)
# Note this is a symbol, not an expression
print(type(first_gate.exponent))

The second object returned by `cirq.flatten` is an object that can map sweeps with particular values to the new symbols that corresond to the expressions, with the value being that of the evaluated expression.

In [ ]:
sweep = cirq.Linspace(a, start=0, stop=3, length=4)
print(f"Old {sweep}")

new_sweep = expr_map.transform_sweep(sweep)
print(f"New {new_sweep}")

One can then use these new sweep elements with the flattened circuit

In [ ]:
for params in new_sweep:
    print(c_flat, '=>', end=' ')
    print(cirq.resolve_parameters(c_flat, params))

Using `cirq.flatten` one can always take a paramterized circuit with complicated expressions plus a sweep, and produce an equivalent circuit with no expressions, only symbols, and a sweep for these symbols. Because this is a common flow, cirq provides `cirq.flatten_sweep` to do this in one step:

In [ ]:
c_flat, new_sweep = cirq.flatten_with_sweep(circuit, sweep)
print(c_flat)
print(new_sweep)

We can then directly use these objects to run the sweeps. For example, here we use them to perform a simulation:

In [ ]:
sim = cirq.Simulator()
results = sim.run_sweep(c_flat, repetitions=20, params=new_sweep)
for result in results:
    print(result.params, result)

Where we see that the different flatten parameters have corresponding different results of the simulation.